# 线性回归的简洁实现

### 1. 生成数据集

In [171]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = torch.tensor(4.2)
features, labels = d2l.synthetic_data(true_w,true_b,1000)

### 2. 读取数据集

我们可以调用框架中现有的API来读取数据。 我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [172]:
def load_array(data_arrays, batch_size, is_train = True):   #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [173]:
next(iter(data_iter))

[tensor([[ 1.6202,  0.4736],
         [ 0.2814, -0.3443],
         [-0.4907, -0.1302],
         [-0.5667, -1.3578],
         [ 0.0396, -1.4646],
         [ 0.2695, -1.0965],
         [-0.5189,  0.5914],
         [ 1.0722, -0.2794],
         [-0.5667, -0.2096],
         [ 0.4675,  1.1573]]),
 tensor([[5.8381],
         [5.9347],
         [3.6553],
         [7.6949],
         [9.2546],
         [8.4559],
         [1.1368],
         [7.3030],
         [3.7762],
         [1.1933]])]

### 3. 定义模型

In [174]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

### 4. 初始化模型参数

深度学习框架通常有预定义的方法来初始化参数。 在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零。

In [175]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### 5. 定义损失函数

In [176]:
loss = nn.MSELoss()

### 6. 定义优化算法

In [177]:
optim = torch.optim.SGD(net.parameters(), lr = 0.03)

### 7. 训练

 对于每一个小批量，我们会进行以下步骤:

- 通过调用net(X)生成预测并计算损失l（前向传播）。

- 通过进行反向传播来计算梯度。

- 通过调用优化器来更新模型参数。

In [178]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        y_hat = net(X)
        l = loss(y_hat, y)
        optim.zero_grad()
        l.backward()
        optim.step()
    #计算总的loss
    with torch.no_grad():
        l = loss(net(features),labels)
        print(f'epoch:{epoch}, loss:{l:.5f}')


epoch:0, loss:0.00020
epoch:1, loss:0.00010
epoch:2, loss:0.00010


In [179]:
w = net[0].weight.data
b = net[0].bias.data
print('w的估计误差: ',(true_w - w).flatten().tolist())
print('b的估计误差: ',(true_b - b).tolist())

w的估计误差:  [0.00015079975128173828, 0.00026297569274902344]
b的估计误差:  [0.000514984130859375]
